In [13]:
import pandas as pd
import numpy as np

In [2]:
books = pd.read_csv('books_cleaned.csv')


In [3]:
books["categories"].value_counts().reset_index().query('count > 50')

,categories,count
0,Fiction,2125
1,Juvenile Fiction,391
2,Biography & Autobiography,312
3,History,208
4,Literary Criticism,124
5,Philosophy,117
6,Religion,117
7,Comics & Graphic Novels,116
8,Drama,91
9,Juvenile Nonfiction,57


In [4]:
category_mapping = {
    'Fiction' : "Fiction",
    'Juvenile Fiction' : "Children's Fiction",
    'Biography & Autobiography' : "Nonfiction",
    'History' : "Nonfiction",
    'Literary Criticistm' : "Nonfiction",
    'Philosophy' : "Nonfiction",
    'Religion' : "Nonfiction",
    'Comics & Graphic Novels' : "Fiction",
    'Drama' : "Fiction",
    'Juvenile Nonfiction' : "Children's Nondfiction",
    'Science' : "Nonfiction",
    'Poetry' : "Fiction"
}

In [5]:
books["simpler_categories"] = books["categories"].map(category_mapping)

In [6]:
books["simpler_categories"].isna().sum()

1587

In [7]:
# df with categories not empty
books[~(books["simpler_categories"].isna())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simpler_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
8,9780006482079,0006482074,Warhost of Vastmark,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,2966.0,Warhost of Vastmark,9780006482079 Tricked once more by his wily ha...,Fiction
30,9780006646006,000664600X,Ocean Star Express,Mark Haddon;Peter Sutton,Juvenile Fiction,http://books.google.com/books/content?id=I2QZA...,Joe and his parents are enjoying a summer holi...,2002.0,3.50,32.0,1.0,Ocean Star Express,9780006646006 Joe and his parents are enjoying...,Children's Fiction
46,9780007121014,0007121016,Taken at the Flood,Agatha Christie,Fiction,http://books.google.com/books/content?id=3gWlx...,A Few Weeks After Marrying An Attractive Young...,2002.0,3.71,352.0,8852.0,Taken at the Flood,9780007121014 A Few Weeks After Marrying An At...,Fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5220,9784770028969,4770028962,Coin Locker Babies,村上龍,Fiction,http://books.google.com/books/content?id=87DJw...,Rescued from the lockers in which they were le...,2002.0,3.75,393.0,5560.0,Coin Locker Babies,9784770028969 Rescued from the lockers in whic...,Fiction
5221,9787506435673,7506435675,Good Fight,Judd Winick;Matthew Clark,Juvenile Fiction,http://books.google.com/books/content?id=xGPZA...,Presents the adventures of a new group of hero...,2007.0,NaN,NaN,NaN,Good Fight,9787506435673 Presents the adventures of a new...,Children's Fiction
5222,9788122200850,8122200850,"Cry, the Peacock",Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,"Cry, the Peacock",9788122200850 This book is the story of a youn...,Fiction
5228,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction


In [8]:
from transformers import pipeline
fiction_categories = ["Fiction", "Nonfiction"]
pipe = pipeline("zero-shot-classification", 
                model="facebook/bart-large-mnli",
                device="cuda")

Device set to use cuda


In [9]:
# !pip install torch==2.0.0+cu117 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install -U accelerate
# !pip install -q --upgrade accelerate einops xformers


In [10]:
sequence = books.loc[books["simpler_categories"] == "Fiction", "description"].reset_index(drop=True)[0]
sequence

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [11]:
pipe(sequence, fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [16]:
max_idx = np.argmax(pipe(sequence, fiction_categories)["scores"])
max_label = pipe(sequence, fiction_categories)["labels"][max_idx]
max_label

'Fiction'

In [31]:
def generate_predictions(sequence, categories):
    predictions = pipe(sequence, categories)
    max_idx = np.argmax(predictions["scores"])
    max_label = predictions["labels"][max_idx]
    return max_label


In [24]:
from tqdm import tqdm
samples_to_test = 100
actual_category = []
predicted_category = []

for i in tqdm(range(samples_to_test)):
    sequence = books.loc[books["simpler_categories"] == "Fiction", 'description'].reset_index(drop = True)[i]
    predicted_category+= [generate_predictions(sequence, fiction_categories)]
    actual_category+= ["Fiction"]

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:22<00:00,  4.44it/s]


In [26]:
from tqdm import tqdm

for i in tqdm(range(samples_to_test)):
    sequence = books.loc[books["simpler_categories"] == "Nonfiction", 'description'].reset_index(drop = True)[i]
    predicted_category+= [generate_predictions(sequence, fiction_categories)]
    actual_category+= ["Nonfiction"]

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:22<00:00,  4.36it/s]


In [27]:
predictions_df = pd.DataFrame({"actual_categories": actual_category, "predicted_categories": predicted_category})

predictions_df["correct_prediction"]=np.where(predictions_df["actual_categories"]==predictions_df["predicted_categories"], 1, 0)

predictions_df["correct_prediction"].sum() / len(predictions_df)

0.815

- Since this is a good accuracy for zero shot classification. Now we will use this to predict the category for all the missing category

In [28]:
isbns = []
predicted_category = []

missing_category = books.loc[books["simpler_categories"].isna(), ["isbn13", "description"]].reset_index(drop=True)

In [32]:
for i in tqdm(range(0, len(missing_category))):
    sequence = missing_category["description"][i]
    predicted_category += [generate_predictions(sequence, fiction_categories)]
    isbns += [missing_category["isbn13"][i]]

100%|██████████████████████████████████████████████████████████████████████████████| 1587/1587 [02:27<00:00, 10.75it/s]


In [33]:
missing_predicted_df = pd.DataFrame({"isbn13":isbns, "predicted_categories": predicted_category})


In [37]:
# since the missing df is a subset of the original dataframe, we should do left join
books = pd.merge(books, missing_predicted_df, on="isbn13", how="left")


In [38]:
#now where the "simpler_categories" data is missing, we can replace with predicted_categories  
books["simpler_categories"]=np.where(books["simpler_categories"].isna(),books["predicted_categories"], books["simpler_categories"])
books = books.drop(columns = ["predicted_categories"]) #drop the column now
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simpler_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction


In [40]:
books["simpler_categories"].value_counts().reset_index()

,simpler_categories,count
0,Fiction,2888
1,Nonfiction,1894
2,Children's Fiction,391
3,Children's Nondfiction,57


In [42]:
#why we did not consider the other categories while calculating accuracy
books[books["categories"].str.lower().isin([
    "romance",
    "science fiction",
    "scifi",
    "fanatasy"
    "horror",
    "mystery"
    "comedy",
    "crime"
])].shape

#they have only 8 rows hence not worth our time

(8, 14)

In [43]:
books.to_csv("books_with_categories.csv",index=False)